## Setup

### Check connection and find available solvers 

(guide: https://docs.ocean.dwavesys.com/en/latest/overview/sapi.html )



In [1]:
from dwave.cloud import Client

In [5]:
with Client.from_config() as client:
    for s in client.get_solvers():
        print(s)

BQMSolver(id='hybrid_binary_quadratic_model_version2')
DQMSolver(id='hybrid_discrete_quadratic_model_version1')
CQMSolver(id='hybrid_constrained_quadratic_model_version1')
StructuredSolver(id='Advantage_system6.1')
StructuredSolver(id='Advantage_system4.1')
StructuredSolver(id='DW_2000Q_6')
StructuredSolver(id='Advantage2_prototype1.1')


In [6]:
with Client.from_config() as client:
    for s in (client.get_solvers(hybrid=True)):
        print(s)

BQMSolver(id='hybrid_binary_quadratic_model_version2')
DQMSolver(id='hybrid_discrete_quadratic_model_version1')
CQMSolver(id='hybrid_constrained_quadratic_model_version1')


In [10]:
with Client.from_config() as client:
    regions = client.get_regions()
    for r in regions:
        print(r)
    for code,info in regions.items():
       print(f"{info['name']} ({code}): {info['endpoint']}")


na-west-1
eu-central-1
North America (na-west-1): https://na-west-1.cloud.dwavesys.com/sapi/v2/
Europe (eu-central-1): https://eu-central-1.cloud.dwavesys.com/sapi/v2/


## A toy problem: Optimize rectangle area + correct settings in a binary circuits

### Step 1: formulate the problem

Toy problem 1: What is the optimal AREA of a rectangle, given that the circumference is fixed to 8? (The answer should be the 2x2 square with area 4)

In [23]:
from dimod import ConstrainedQuadraticModel, Integer
i = Integer('i', upper_bound=4) # The 'lenght' of the rectangle
j = Integer('j', upper_bound=4) # The 'width' of the rectangle  
cqm = ConstrainedQuadraticModel()
cqm.set_objective(-i*j)  # Objective: maximize the area 
cqm.add_constraint(2*i+2*j <= 8, "Max perimeter") # Constraint: perimiter restricted to 8 units
print(cqm)

Toy problem 2: what are consistent in- and outputs of common logical gates like AND, OR, NOT, etc? 

D-Wave automatically translates these to a QUBO. 

In [60]:
from dimod.generators import and_gate
bqm = and_gate('in1', 'in2', 'out')
print(bqm)

BinaryQuadraticModel({'in1': 0.0, 'in2': 0.0, 'out': 3.0}, {('in2', 'in1'): 1.0, ('out', 'in1'): -2.0, ('out', 'in2'): -2.0}, 0.0, 'BINARY')


### Step 2: Send to a solver

NB: For the NOT gate, there are four correct results:

00; 0

01; 0

10; 0

11; 1

We should receive any of these at random. Let's see if we find them with equal probability?

In [24]:
from dimod.generators import and_gate
from dwave.system import LeapHybridSampler
bqm = and_gate('x1', 'x2', 'y1')
sampler = LeapHybridSampler()    
answer = sampler.sample(bqm)   
print(answer)    

  x1 x2 y1 energy num_oc.
0  1  0  0    0.0       1
['BINARY', 1 rows, 1 samples, 3 variables]


In [50]:
answers = []   # Set up a new array in which we store results

In [51]:
# Run 10 times (NB: this costs quite some time!)
for j in range(10):
    answers.append(sampler.sample(bqm) )

In [54]:
frequency_counter = dict()
for ans in answers:
    for d in ans.samples():
        d = str(d)
        if d in frequency_counter:
            frequency_counter[d] += 1
        else :
            frequency_counter[d] = 1

frequency_counter

{"{'x1': 1, 'x2': 0, 'y1': 0}": 4,
 "{'x1': 1, 'x2': 1, 'y1': 1}": 3,
 "{'x1': 0, 'x2': 1, 'y1': 0}": 3}

## Problem inspection

### Sampling from Ising Models

Use DWaveSampler to encode an explicit Ising Model.

Edges must be compatible with the Chimera graph. For example, edge (0,1) is okay, but edge (0,2) will cause an error. 

In [63]:
from dwave.system import DWaveSampler
import dwave.inspector

# Get solver
sampler = DWaveSampler(solver=dict(qpu=True))

# Here we define a simple Ising model
h = {}
J = {(0, 4): 1, (0, 5): 1, (1, 4): 1, (1, 5): -1, (2,4): 1}

# Double check that all 'edge labels' that we use are compatible with the solver?
print( all(edge in sampler.edgelist for edge in J) )

# Sample
response = sampler.sample_ising(h, J, num_reads=100)

# Inspect
dwave.inspector.show(response)   

True


'http://127.0.0.1:18000/?problemId=606580b6-789b-48cc-b81b-e239c74da504'

### Solving logical problems

In [78]:
from dwave.system import DWaveSampler, EmbeddingComposite

# Take our good old AND Gate:
bqm = and_gate('x1', 'x2', 'y1')

# EmbeddingComposite is supposed to automatically map the problem to the QPU's graph. 
sampler = EmbeddingComposite(DWaveSampler(solver=dict(qpu=True)))


sampleset = sampler.sample(bqm, num_reads=100, chain_strength=0.1)

for j in sampleset.data():
    print(j)

Sample(sample={'x1': 0, 'x2': 1, 'y1': 0}, energy=0.0, num_occurrences=23, chain_break_fraction=0.0)
Sample(sample={'x1': 0, 'x2': 0, 'y1': 0}, energy=0.0, num_occurrences=28, chain_break_fraction=0.0)
Sample(sample={'x1': 1, 'x2': 0, 'y1': 0}, energy=0.0, num_occurrences=26, chain_break_fraction=0.0)
Sample(sample={'x1': 1, 'x2': 1, 'y1': 1}, energy=0.0, num_occurrences=23, chain_break_fraction=0.0)


In [77]:
dwave.inspector.show(sampleset)

'http://127.0.0.1:18000/?problemId=713836cb-8fd3-48a7-b136-a4b7c0bd2158'

The results are medioce: the vast majority of results gave energy E=1 instead of the optimal E=0. Let's vary parameters to get better solutions. 

Solutions:

chain strength = 0.1 (most samples are wrong)

chain_strenght = 0.7 (all samples are correct)

*EDIT*: Re-running gives 100% score even on 0.1 chain strength. To be investigated later. 